In [ ]:
import networkx as nx
import pandas as pd
from operator import itemgetter
import matplotlib.pyplot as plt


In [ ]:
poj_G = nx.read_edgelist('./nx_create/nx_graph_df_poj.csv'
                     , create_using=nx.DiGraph()
                     , nodetype=int
                     , data=[('weight', int)]
                     )




In [ ]:
list(poj_G.edges(data=True))

In [ ]:
len(list(poj_G.edges(data=True)))

In [ ]:
from tqdm import tqdm
node_dict_degree_clus = {}
pr = nx.pagerank(poj_G)

degree_centralityG = nx.degree_centrality(poj_G)

eigenvector_centralityG = nx.eigenvector_centrality(poj_G,weight='weight')

average_degree_connectivityG = nx.average_degree_connectivity(poj_G, weight="weight")

harmonic_centralityG = nx.harmonic_centrality(poj_G,distance='weight')

betweenness_centralityG = nx.betweenness_centrality(poj_G,weight='weight')

for node_id in tqdm(nx.nodes(poj_G)):
    metadata = {}
    metadata['PageRank'] = round(pr[node_id],5)
    metadata['DegreeCentrality'] = round(degree_centralityG[node_id],5)#度中心性
    metadata['EigenvectorCentrality'] = round(eigenvector_centralityG[node_id],5)#特征向量中心度
    metadata['HarmonicCentrality'] = round(harmonic_centralityG[node_id],5)#紧密中心性 closses
    metadata['BetweennessCentrality'] = round(betweenness_centralityG[node_id],5)#介数中心性 betweenness
    ego = nx.ego_graph(poj_G, node_id, distance='weight')
    metadata['ClusteringCoeff'] = round(nx.average_clustering(ego,weight='weight'),5)#聚类系数
    node_dict_degree_clus[node_id] = metadata

   

In [ ]:
import pickle
pickle.dump(node_dict_degree_clus, open( 'poj_node_dict_Centrality_clus_single_features.pkl', 'wb'))   

In [ ]:
node_dict_degree_clus


In [ ]:

poj_sna = []
for item_id,metadata in tqdm(node_dict_degree_clus.items()):
    singledata = {}
    singledata['item_id'] = item_id
    singledata['PageRank'] = metadata['PageRank']
    singledata['DegreeCentrality'] = metadata['DegreeCentrality']
    singledata['EigenvectorCentrality'] = metadata['EigenvectorCentrality']
    singledata['HarmonicCentrality'] = metadata['HarmonicCentrality']
    singledata['BetweennessCentrality'] = metadata['BetweennessCentrality']
    singledata['ClusteringCoeff'] = metadata['ClusteringCoeff']
    poj_sna.append(singledata)

In [ ]:
poj_sna_analysis = pd.DataFrame(poj_sna)
poj_sna_analysis_item_id = poj_sna_analysis.set_index('item_id')
poj_sna_analysis_item_id

In [ ]:
import numpy as np
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))

poj_sna_analysis['HarmonicCentrality'] = poj_sna_analysis[['HarmonicCentrality']].apply(max_min_scaler)
poj_sna_analysis_item_id['HarmonicCentrality'] = poj_sna_analysis_item_id[['HarmonicCentrality']].apply(max_min_scaler)


##
poj_sna_analysis['PageRank'] = poj_sna_analysis[['PageRank']].apply(max_min_scaler)
poj_sna_analysis_item_id['PageRank'] = poj_sna_analysis_item_id[['PageRank']].apply(max_min_scaler)

##
poj_sna_analysis['DegreeCentrality'] = poj_sna_analysis[['DegreeCentrality']].apply(max_min_scaler)
poj_sna_analysis_item_id['DegreeCentrality'] = poj_sna_analysis_item_id[['DegreeCentrality']].apply(max_min_scaler)
##

poj_sna_analysis['EigenvectorCentrality'] = poj_sna_analysis[['EigenvectorCentrality']].apply(max_min_scaler)
poj_sna_analysis_item_id['EigenvectorCentrality'] = poj_sna_analysis_item_id[['EigenvectorCentrality']].apply(max_min_scaler)
#
poj_sna_analysis['ClusteringCoeff'] = poj_sna_analysis[['ClusteringCoeff']].apply(max_min_scaler)
poj_sna_analysis_item_id['ClusteringCoeff'] = poj_sna_analysis_item_id[['ClusteringCoeff']].apply(max_min_scaler)
#
# BetweennessCentrality
poj_sna_analysis['BetweennessCentrality'] = poj_sna_analysis[['BetweennessCentrality']].apply(max_min_scaler)
poj_sna_analysis_item_id['BetweennessCentrality'] = poj_sna_analysis_item_id[['BetweennessCentrality']].apply(max_min_scaler)
##

In [ ]:
from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=1)

In [ ]:
# neigh.fit(poj_sna_analysis[['PageRank','DegreeCentrality','EigenvectorCentrality','HarmonicCentrality','BetweennessCentrality','ClusteringCoeff']].values)
neigh.fit(poj_sna_analysis[['PageRank']].values)

In [ ]:
#pagerank

recall_item_num = 11
poj = pd.read_csv('./nx_create/nx_graph_df_poj.csv',header=None,sep=' ',names=['source','target','weight'])
# poj = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
val_source_target = []
for _ ,group in poj.groupby('source'):
#     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
    tmp = group.iloc[0,:]
    val_source_target.append((tmp.source,tmp.target))
val_source_target
source_target = list(zip(poj.source.tolist(),poj.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(poj_sna_analysis['item_id'].unique()):
    ind = neigh.kneighbors(poj_sna_analysis_item_id.loc[iid,'PageRank'].reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = poj_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[:]
#     print(item_recall_list)

# eva_data = []
# for iid,group in poj.groupby('source'):
#     if len(group)>10:
#         tmp = list(zip(group.source,group.target))[:10]
#     else: tmp = list(zip(group.source,group.target))
#     eva_data.extend(tmp[:])   


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        



In [ ]:
# demo[1001],demo[1000],demo[2056]

In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)

In [ ]:
#DegreeCentrality

neigh1 = NearestNeighbors(n_neighbors=1)
neigh1.fit(poj_sna_analysis[['DegreeCentrality']].values)

In [ ]:
#pagerank

recall_item_num = 11
poj = pd.read_csv('./nx_create/nx_graph_df_poj.csv',header=None,sep=' ',names=['source','target','weight'])
# poj = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
# val_source_target = []
# for _ ,group in poj.groupby('source'):
# #     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
#     tmp = group.iloc[0,:]
#     val_source_target.append((tmp.source,tmp.target))
# val_source_target
source_target = list(zip(poj.source.tolist(),poj.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(poj_sna_analysis['item_id'].unique()):
    ind = neigh1.kneighbors(poj_sna_analysis_item_id.loc[iid,'DegreeCentrality'].reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = poj_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[:]


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    
    ###
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        



In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)

In [ ]:
#EigenvectorCentrality
neigh2 = NearestNeighbors(n_neighbors=1)
neigh2.fit(poj_sna_analysis[['EigenvectorCentrality']].values)
   

In [ ]:
#pagerank

recall_item_num = 11
poj = pd.read_csv('./nx_create/nx_graph_df_poj.csv',header=None,sep=' ',names=['source','target','weight'])
# poj = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
# val_source_target = []
# for _ ,group in poj.groupby('source'):
# #     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
#     tmp = group.iloc[0,:]
#     val_source_target.append((tmp.source,tmp.target))
# val_source_target
source_target = list(zip(poj.source.tolist(),poj.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(poj_sna_analysis['item_id'].unique()):
    ind = neigh2.kneighbors(poj_sna_analysis_item_id.loc[iid,'EigenvectorCentrality'].reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = poj_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[:]


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    
    ###
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        



In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)

In [ ]:
# HarmonicCentrality
neigh3 = NearestNeighbors(n_neighbors=1)
neigh3.fit(poj_sna_analysis[['HarmonicCentrality']].values)

In [ ]:


recall_item_num = 11
poj = pd.read_csv('./nx_create/nx_graph_df_poj.csv',header=None,sep=' ',names=['source','target','weight'])
# poj = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
# val_source_target = []
# for _ ,group in poj.groupby('source'):

# #     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
#     tmp = group.iloc[0,:]
#     val_source_target.append((tmp.source,tmp.target))
# val_source_target
source_target = list(zip(poj.source.tolist(),poj.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(poj_sna_analysis['item_id'].unique()):
    ind = neigh3.kneighbors(poj_sna_analysis_item_id.loc[iid,'HarmonicCentrality'].reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = poj_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[:]


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    
    ###
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        



In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)
    

In [ ]:
metrics_recall(source_target,item_recall_list,10)


In [ ]:
# 4'BetweennessCentrality','ClusteringCoeff'
neigh4 = NearestNeighbors(n_neighbors=1)
neigh4.fit(poj_sna_analysis[['BetweennessCentrality']].values)


In [ ]:


recall_item_num = 11
poj = pd.read_csv('./nx_create/nx_graph_df_poj.csv',header=None,sep=' ',names=['source','target','weight'])
# poj = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
# val_source_target = []
# for _ ,group in poj.groupby('source'):
# #     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
#     tmp = group.iloc[0,:]
#     val_source_target.append((tmp.source,tmp.target))
# val_source_target
source_target = list(zip(poj.source.tolist(),poj.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(poj_sna_analysis['item_id'].unique()):
    ind = neigh4.kneighbors(poj_sna_analysis_item_id.loc[iid,'BetweennessCentrality'].reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = poj_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[:]


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    
    ###
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        



In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)

In [ ]:
# 4'BetweennessCentrality','ClusteringCoeff'
neigh5 = NearestNeighbors(n_neighbors=1)
neigh5.fit(poj_sna_analysis[['ClusteringCoeff']].values)

In [ ]:


recall_item_num = 11
poj = pd.read_csv('./nx_create/nx_graph_df_poj.csv',header=None,sep=' ',names=['source','target','weight'])
# poj = pd.read_csv('./nx_create/clean__data_full_.txt',sep='\t')
# val_source_target = []
# for _ ,group in poj.groupby('source'):
# #     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
#     tmp = group.iloc[0,:]
#     val_source_target.append((tmp.source,tmp.target))
# val_source_target
source_target = list(zip(poj.source.tolist(),poj.target.tolist()))
# source_target_dict = {}

item_recall_list = {}
for iid in tqdm(poj_sna_analysis['item_id'].unique()):
    ind = neigh5.kneighbors(poj_sna_analysis_item_id.loc[iid,'ClusteringCoeff'].reshape((1,-1)),recall_item_num, return_distance=False)
    item_recall_list[iid] = poj_sna_analysis.loc[ind.tolist()[0]].iloc[:,0].tolist()[:]


def metrics_recall(eva_data,item_recall_list, k=10):
    item_num = len(item_recall_list)
    
    hit_num = 0
    score=0
    past = []
    for iid, recall_list in tqdm(item_recall_list.items()):#从item列表中读取每一个item
        # 获取前k个召回的结果
        tmp_recall_items = item_recall_list[iid][:k] #返回一个item的列表
        for s,t in eva_data:
            if s == iid and t in list(tmp_recall_items):#在评估数据中查找是否存在推荐列表中
#                 print("标注：",s,t)
                rank = list(tmp_recall_items).index(t)
                score += 1.0/(rank+1.0)
                hit_num += 1
                break
    
    ###
    

    ###
            
    mrr_score = round(score* 1.0 / item_num, 5)
    hit_rate = round(hit_num * 1.0 / item_num, 5)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', item_num)
    print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'mrr_score: ', mrr_score, 'user_num : ', item_num)
        
        



In [ ]:
metrics_recall(source_target,item_recall_list,3)

In [ ]:
metrics_recall(source_target,item_recall_list,5)

In [ ]:
metrics_recall(source_target,item_recall_list,10)

In [ ]:
poj = pd.read_csv('./nx_create/nx_graph_df_poj.csv',header=None,sep=' ',names=['source','target','weight'])
val_source_target = []
for _ ,group in poj.groupby('source'):
#     tmp = group.iloc[np.random.choice(range(len(group)),1)[0],:]
    tmp = group.iloc[0,:]
    val_source_target.append((tmp.source,tmp.target))
val_source_target

In [ ]:
poj

In [ ]:
iid